In [7]:
import ee
ee.Authenticate(force=True)
ee.Initialize(project='ee-prathama76')

Enter verification code: 4/1AQSTgQHBBZVP8LydwOxbvudee871gjCaDuna5cO7FyIwskbC2L_PvZaKiL0

Successfully saved authorization token.


In [8]:
import ee
import pandas as pd

# Initialize the Earth Engine API
ee.Initialize()

# Define the function to fetch SAR data for a given point
def fetch_sar_data_for_point(lat, lon, crop_name, start_date, end_date):
    # Define the point of interest
    point = ee.Geometry.Point(lon, lat)
    
    # Load the Sentinel-1 ImageCollection
    sentinel1 = ee.ImageCollection("COPERNICUS/S1_GRD") \
        .filterBounds(point) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.eq("instrumentMode", "IW")) \
        .filter(ee.Filter.inList("orbitProperties_pass", ["ASCENDING", "DESCENDING"])) \
        .select(["VV", "VH"])
    
    # Log the number of images found for the point
    num_images = sentinel1.size().getInfo()
    print(f"Processing point ({lat}, {lon}) - Crop: {crop_name} - Images found: {num_images}")
    
    if num_images == 0:
        print(f"No data found for point ({lat}, {lon}) within the specified time range.")
        return []
    
    # Extract data for the point
    def extract_values(image):
        # Get the mean value for each band at the point
        value = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=point,
            scale=10
        )
        
        # Get the orbit pass direction (ascending or descending)
        orbit_pass = image.get("orbitProperties_pass")
        
        return ee.Feature(None, {
            "date": image.date().format("YYYY-MM-dd"),
            "VV": value.get("VV"),
            "VH": value.get("VH"),
            "Orbit": orbit_pass,
            "Crop_Name": crop_name
        })
    
    # Map the extraction function to the ImageCollection
    features = sentinel1.map(extract_values).getInfo()["features"]
    
    # Convert features to a list of dictionaries
    data = []
    for feature in features:
        props = feature["properties"]
        data.append({
            "date": props["date"],
            "VV": props["VV"],
            "VH": props["VH"],
            "Orbit": props["Orbit"],  # Include orbit pass direction
            "Crop_Name": props["Crop_Name"],
            "latitude": lat,
            "longitude": lon
        })
    
    return data

# Read the input CSV file
input_csv = "Karnataka_Datasets/Agrifield.csv"  # Replace with your input file path
df = pd.read_csv(input_csv)

# Define the time range for Sentinel-1 data
start_date = "2021-04-01"
end_date = "2021-10-31"

# Initialize an empty list to store results
all_data = []

# Process each row in the CSV
for index, row in df.iterrows():
    lat = row["Latitude"]
    lon = row["Longitude"]
    crop_name = row["Crop_Name"]
    
    # Fetch SAR data for the point
    try:
        sar_data = fetch_sar_data_for_point(lat, lon, crop_name, start_date, end_date)
        if sar_data:
            all_data.extend(sar_data)
        else:
            print(f"No data returned for point ({lat}, {lon})")
    except Exception as e:
        print(f"Error fetching data for point ({lat}, {lon}): {e}")

# Convert all data to a DataFrame
output_df = pd.DataFrame(all_data)

# Check if the DataFrame is empty
if output_df.empty:
    print("No SAR data was retrieved for the given points and time range.")
else:
    # Save the output to a new CSV file
    output_csv = "Karnataka_Datasets/Agrifield_SAR.csv"  # Replace with your desired output file path
    output_df.to_csv(output_csv, index=False)
    print(f"Data successfully saved to {output_csv}")


Processing point (19.286188981169765, 83.39884788522168) - Crop: No Crop/Fallow - Images found: 18
Processing point (19.28218382824294, 83.40888769570374) - Crop: No Crop/Fallow - Images found: 18
Processing point (19.274077490545768, 83.41135111694561) - Crop: No Crop/Fallow - Images found: 18
Processing point (25.576379322808634, 87.44469489298854) - Crop: No Crop/Fallow - Images found: 53
Processing point (25.58443108803743, 87.44496037785558) - Crop: Maize - Images found: 53
Processing point (25.57874959821723, 87.44691683244083) - Crop: Maize - Images found: 53
Processing point (25.58113095880816, 87.43958604646521) - Crop: Potato - Images found: 53
Processing point (25.580502907101057, 87.43849983973524) - Crop: Potato - Images found: 53
Processing point (25.4156385365353, 76.5238002207973) - Crop: Wheat - Images found: 25
Processing point (25.41395828410415, 76.52440404234842) - Crop: Wheat - Images found: 25
Processing point (27.48601319963976, 82.47641687911434) - Crop: Wheat 

Processing point (27.6462309474625, 82.17384099646662) - Crop: No Crop/Fallow - Images found: 53
Processing point (27.646298419479454, 82.17653518580248) - Crop: No Crop/Fallow - Images found: 53
Processing point (27.64596290164912, 82.17713880065948) - Crop: No Crop/Fallow - Images found: 53
Processing point (27.64537628455708, 82.17750100044321) - Crop: No Crop/Fallow - Images found: 53
Processing point (27.645131062343246, 82.17353678486629) - Crop: No Crop/Fallow - Images found: 53
Processing point (27.64491905307684, 82.17942776666509) - Crop: No Crop/Fallow - Images found: 53
Processing point (27.64458743342521, 82.17988908344867) - Crop: No Crop/Fallow - Images found: 53
Processing point (27.6443025338183, 82.17962270288687) - Crop: No Crop/Fallow - Images found: 53
Processing point (27.64124192176936, 82.17476144687274) - Crop: No Crop/Fallow - Images found: 53
Processing point (27.637171147758487, 82.16786448327511) - Crop: No Crop/Fallow - Images found: 53
Processing point (2

Processing point (24.656034432675348, 76.89987247839125) - Crop: Mustard - Images found: 42
Processing point (27.58440682447113, 81.39683464825765) - Crop: Wheat - Images found: 53
Processing point (27.58312168863517, 81.3895427089092) - Crop: Wheat - Images found: 53
Processing point (27.57335030707789, 81.40127780621933) - Crop: Wheat - Images found: 53
Processing point (27.57137174734545, 81.39880525443225) - Crop: Wheat - Images found: 53
Processing point (27.57118644058443, 81.39530456682954) - Crop: Wheat - Images found: 53
Processing point (27.56757516411569, 81.39505091462894) - Crop: Wheat - Images found: 53
Processing point (27.56724786691598, 81.39377718962005) - Crop: Wheat - Images found: 53
Processing point (27.58700455260593, 81.39069845702882) - Crop: Mustard - Images found: 53
Processing point (27.58324614547648, 81.39810264927176) - Crop: Mustard - Images found: 53
Processing point (27.58231168667588, 81.40213650536317) - Crop: Mustard - Images found: 53
Processing po

Processing point (25.22210341023104, 76.32949425883491) - Crop: Wheat - Images found: 25
Processing point (25.20172503119954, 76.33793015989947) - Crop: Wheat - Images found: 25
Processing point (27.6092199537593, 82.26629789368987) - Crop: Wheat - Images found: 53
Processing point (24.810229560954834, 76.78114623698558) - Crop: Wheat - Images found: 42
Processing point (25.3098230198812, 76.38591369644475) - Crop: Wheat - Images found: 25
Processing point (25.308733597508496, 76.3903621222844) - Crop: Wheat - Images found: 25
Processing point (25.303801786094034, 76.37734090314751) - Crop: Mustard - Images found: 25
Processing point (25.04952283212902, 76.39670543893993) - Crop: Mustard - Images found: 25
Processing point (27.97394355019707, 81.49594379166298) - Crop: Wheat - Images found: 53
Processing point (27.973169435183635, 81.49489549790952) - Crop: Wheat - Images found: 53
Processing point (25.51083545343984, 87.60345891742477) - Crop: Wheat - Images found: 35
Processing point

Processing point (25.38993385036056, 87.7321248396823) - Crop: Maize - Images found: 35
Processing point (24.642751543646654, 76.78139036414888) - Crop: Garlic - Images found: 42
Processing point (25.46123482897323, 87.80067155288421) - Crop: Mustard - Images found: 35
Processing point (25.46001495025904, 87.80032689400308) - Crop: Mustard - Images found: 35
Processing point (25.479379966710464, 87.79170900708695) - Crop: Rice - Images found: 35
Processing point (25.46024011673578, 87.81232330533295) - Crop: Rice - Images found: 35
Processing point (27.75268681607012, 81.53716677010726) - Crop: Wheat - Images found: 53
Processing point (27.75170169605058, 81.53241740429182) - Crop: Wheat - Images found: 53
Processing point (27.75040469811057, 81.53462025099782) - Crop: Wheat - Images found: 53
Processing point (27.75191882536234, 81.53315078083925) - Crop: Lentil - Images found: 53
Processing point (27.75039735130505, 81.5359754887277) - Crop: Lentil - Images found: 53
Processing point

...

Processing point (27.53468072723847, 82.15292817733359) - Crop: No Crop/Fallow - Images found: 53
Processing point (27.53447739503872, 82.15255666914848) - Crop: No Crop/Fallow - Images found: 53
Processing point (27.53426558209158, 82.15317730913466) - Crop: No Crop/Fallow - Images found: 53
Processing point (27.723526453057467, 81.39360401543162) - Crop: Mustard - Images found: 53
Processing point (25.62181185000171, 87.43592178420865) - Crop: Mustard - Images found: 53
Processing point (25.62195922527797, 87.44552385561555) - Crop: Mustard - Images found: 53
Processing point (25.61539261604772, 87.44702244896853) - Crop: Mustard - Images found: 53
Processing point (25.61675077720497, 87.43754717038674) - Crop: No Crop/Fallow - Images found: 53
Processing point (25.615593183033013, 87.44725351195595) - Crop: Maize - Images found: 53
Processing point (25.611574284255106, 87.45815919700019) - Crop: Maize - Images found: 53
Processing point (19.52395855487909, 83.46992912605197) - Crop:

Processing point (27.646865155309506, 82.29764413506838) - Crop: No Crop/Fallow - Images found: 53
Processing point (27.195958473844133, 81.49682558238064) - Crop: Wheat - Images found: 53
Processing point (27.203766870666765, 81.50147854640534) - Crop: Mustard - Images found: 53
Processing point (27.19627437432309, 81.49805266199054) - Crop: Mustard - Images found: 53
Processing point (27.194833743434312, 81.49590272474894) - Crop: No Crop/Fallow - Images found: 53
Processing point (27.19649592692578, 81.49676994491867) - Crop: Green pea - Images found: 53
Processing point (24.81157437482049, 76.63588894697251) - Crop: Wheat - Images found: 42
Processing point (24.81377204684305, 76.6309477105294) - Crop: Mustard - Images found: 42
Processing point (24.81265272332541, 76.63840330801608) - Crop: Mustard - Images found: 42
Processing point (24.80652091854323, 76.63335171332335) - Crop: Mustard - Images found: 42
Processing point (24.80984501812543, 76.63316263461753) - Crop: Garlic - Im

Processing point (27.748882276166, 82.1570281705082) - Crop: Wheat - Images found: 53
Processing point (27.749482659251257, 82.15728725296508) - Crop: Lentil - Images found: 53
Processing point (27.748379438985445, 82.15761534146856) - Crop: No Crop/Fallow - Images found: 53
Processing point (27.580414541706947, 82.27155341147103) - Crop: Lentil - Images found: 53
Processing point (27.56179125066216, 82.28379077306215) - Crop: Lentil - Images found: 53
Processing point (27.573669394773443, 82.28511524877504) - Crop: No Crop/Fallow - Images found: 53
Processing point (25.668135344417426, 87.47057370815892) - Crop: No Crop/Fallow - Images found: 53
Processing point (25.642869867792022, 87.90436192618598) - Crop: Maize - Images found: 35
Processing point (25.64278492708864, 87.90360716082313) - Crop: Maize - Images found: 35
Processing point (27.916976635693448, 81.48793188792399) - Crop: Wheat - Images found: 53
Processing point (27.916691948620983, 81.49408983625474) - Crop: Wheat - Ima

Processing point (27.84964510327372, 81.34756059898173) - Crop: Wheat - Images found: 53
Processing point (27.84911429011566, 81.35600062079895) - Crop: Wheat - Images found: 53
Processing point (27.849035779353954, 81.34466577959004) - Crop: Wheat - Images found: 53
Processing point (27.847917625934084, 81.35352645432458) - Crop: Wheat - Images found: 53
Processing point (27.847733353981244, 81.35225037300685) - Crop: Wheat - Images found: 53
Processing point (27.84633009680983, 81.34699001563622) - Crop: Wheat - Images found: 53
Processing point (27.845997607249, 81.35309969380934) - Crop: Wheat - Images found: 53
Processing point (27.84591122729362, 81.35129702979687) - Crop: Wheat - Images found: 53
Processing point (27.84437607930517, 81.35635630504325) - Crop: Wheat - Images found: 53
Processing point (27.84430988241835, 81.36122556884777) - Crop: Wheat - Images found: 53
Processing point (27.84317015226497, 81.35825597554754) - Crop: Wheat - Images found: 53
Processing point (27

Processing point (19.0862369097799, 83.5400269805388) - Crop: No Crop/Fallow - Images found: 18
Processing point (19.08556298924849, 83.54032663101594) - Crop: No Crop/Fallow - Images found: 18
Processing point (19.085232986268373, 83.54087970543357) - Crop: No Crop/Fallow - Images found: 18
Processing point (19.0835596002816, 83.54039989065495) - Crop: No Crop/Fallow - Images found: 18
Processing point (19.083466524219588, 83.54279778596) - Crop: No Crop/Fallow - Images found: 18
Processing point (24.747577533108664, 76.69466039627375) - Crop: Wheat - Images found: 42
Processing point (24.743962155457915, 76.68932205447292) - Crop: Mustard - Images found: 42
Processing point (27.23462433529524, 81.56936220572851) - Crop: Mustard - Images found: 53
Processing point (27.18961326674866, 82.30275050610857) - Crop: No Crop/Fallow - Images found: 53
Processing point (25.192919242368227, 76.28064571447082) - Crop: Garlic - Images found: 25
Data successfully saved to Karnataka_Datasets/Agrifi

In [2]:
import os
import ee
import pandas as pd

# Initialize the Earth Engine API
ee.Initialize()

# Define the function to fetch SAR data for a given point
def fetch_sar_data_for_point(lat, lon, crop_name, start_date, end_date):
    # Define the point of interest
    point = ee.Geometry.Point(lon, lat)

    # Load the Sentinel-1 ImageCollection for ascending and descending passes
    sentinel1_asc = ee.ImageCollection("COPERNICUS/S1_GRD") \
        .filterBounds(point) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.eq("instrumentMode", "IW")) \
        .filter(ee.Filter.eq("orbitProperties_pass", "ASCENDING")) \
        .select(["VV", "VH"])

    sentinel1_desc = ee.ImageCollection("COPERNICUS/S1_GRD") \
        .filterBounds(point) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.eq("instrumentMode", "IW")) \
        .filter(ee.Filter.eq("orbitProperties_pass", "DESCENDING")) \
        .select(["VV", "VH"])

    def extract_values(image, pass_direction):
        # Get the mean value for each band at the point
        value = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=point,
            scale=10
        )
        return ee.Feature(None, {
            "date": image.date().format("YYYY-MM-dd"),
            "VV": value.get("VV"),
            "VH": value.get("VH"),
            "Crop_Name": crop_name,
            "Pass_Direction": pass_direction
        })

    # Map the extraction function to each ImageCollection
    asc_features = sentinel1_asc.map(lambda img: extract_values(img, "ASCENDING")).getInfo()["features"]
    desc_features = sentinel1_desc.map(lambda img: extract_values(img, "DESCENDING")).getInfo()["features"]

    # Combine features from ascending and descending passes
    data = []
    for feature in asc_features + desc_features:
        props = feature["properties"]
        data.append({
            "date": props["date"],
            "VV": props["VV"],
            "VH": props["VH"],
            "Crop_Name": props["Crop_Name"],
            "Pass_Direction": props["Pass_Direction"],
            "latitude": lat,
            "longitude": lon
        })

    return data

# Define the time range for Sentinel-1 data
start_date = "2021-04-01"
end_date = "2021-09-30"

# Define the input and output directories
input_dir = "Karnataka_Datasets/Krishnagiri/Clean//"  # Replace with your input directory path
output_dir = "Karnataka_Datasets/Krishnagiri/Clean/"  # Replace with your output directory path

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Process each CSV file in the directory
for file_name in os.listdir(input_dir):
    if file_name.endswith(".csv"):
        input_csv = os.path.join(input_dir, file_name)
        output_csv = os.path.join(output_dir, f"Processed_{file_name}")

        # Log the current file being processed
        print(f"Processing file: {file_name}")

        # Read the input CSV file
        df = pd.read_csv(input_csv)

        # Initialize an empty list to store results
        all_data = []

        # Process each row in the CSV
        for index, row in df.iterrows():
            lat = row["latitude"]
            lon = row["longitude"]
            crop_name = row["Cropname"]

            # Log the current point being processed
            print(f"Processing point ({lat}, {lon}) - Crop: {crop_name}")

            # Fetch SAR data for the point
            try:
                sar_data = fetch_sar_data_for_point(lat, lon, crop_name, start_date, end_date)
                if sar_data:
                    all_data.extend(sar_data)
                    print(f"Data fetched for point ({lat}, {lon}): {len(sar_data)} records")
                else:
                    print(f"No data returned for point ({lat}, {lon})")
            except Exception as e:
                print(f"Error fetching data for point ({lat}, {lon}): {e}")

        # Convert all data to a DataFrame
        output_df = pd.DataFrame(all_data)

        # Check if the DataFrame is empty
        if output_df.empty:
            print(f"No SAR data was retrieved for file: {file_name}")
        else:
            # Save the output to a new CSV file
            output_df.to_csv(output_csv, index=False)
            print(f"Data successfully saved to {output_csv}")


Processing file: Krishnagiri_cleaned_file.csv
Processing point (12.7982671, 77.8027552) - Crop: Redgram
Data fetched for point (12.7982671, 77.8027552): 15 records
Processing point (12.7955541, 77.7950533) - Crop: Eucalyptus
Data fetched for point (12.7955541, 77.7950533): 15 records
Processing point (12.7955571, 77.7950435) - Crop: Eucalyptus
Data fetched for point (12.7955571, 77.7950435): 15 records
Processing point (12.7578162, 77.7718266) - Crop: Ragi
Data fetched for point (12.7578162, 77.7718266): 15 records
Processing point (12.7577922, 77.7718393) - Crop: Ragi
Data fetched for point (12.7577922, 77.7718393): 15 records
Processing point (12.7577921, 77.7718437) - Crop: Ragi
Data fetched for point (12.7577921, 77.7718437): 15 records
Processing point (12.7494526, 77.7865336) - Crop: Coconut
Data fetched for point (12.7494526, 77.7865336): 15 records
Processing point (12.7494504, 77.7865315) - Crop: Coconut
Data fetched for point (12.7494504, 77.7865315): 15 records
Processing po

Data fetched for point (12.7241433, 77.7688367): 15 records
Processing point (12.7550893, 77.7836619) - Crop: Rose
Data fetched for point (12.7550893, 77.7836619): 15 records
Processing point (12.755089, 77.7836622) - Crop: Rose
Data fetched for point (12.755089, 77.7836622): 15 records
Processing point (12.7256886, 77.7674321) - Crop: Ragi
Data fetched for point (12.7256886, 77.7674321): 15 records
Processing point (12.7619276, 77.7830604) - Crop: Cabbage
Data fetched for point (12.7619276, 77.7830604): 15 records
Processing point (12.7619294, 77.7830604) - Crop: Cabbage
Data fetched for point (12.7619294, 77.7830604): 15 records
Processing point (12.7619792, 77.7830624) - Crop: Cabbage
Data fetched for point (12.7619792, 77.7830624): 15 records
Processing point (12.8018465, 77.8018516) - Crop: Redgram
Data fetched for point (12.8018465, 77.8018516): 15 records
Processing point (12.7982377, 77.8061577) - Crop: Ragi
Data fetched for point (12.7982377, 77.8061577): 15 records
Processing

Data fetched for point (12.7534232, 77.7912497): 15 records
Processing point (12.75345128, 77.79114095) - Crop: Avare
Data fetched for point (12.75345128, 77.79114095): 15 records
Processing point (12.7479428, 77.7899906) - Crop: Eucalyptus
Data fetched for point (12.7479428, 77.7899906): 15 records
Processing point (12.7479805, 77.7899015) - Crop: Eucalyptus
Data fetched for point (12.7479805, 77.7899015): 15 records
Processing point (12.714985, 77.767805) - Crop: Banana
Data fetched for point (12.714985, 77.767805): 15 records
Processing point (12.7549294, 77.7845756) - Crop: Avare
Data fetched for point (12.7549294, 77.7845756): 15 records
Processing point (12.7549282, 77.784577) - Crop: Avare
Data fetched for point (12.7549282, 77.784577): 15 records
Processing point (12.7549272, 77.7845754) - Crop: Avare
Data fetched for point (12.7549272, 77.7845754): 15 records
Processing point (12.7931752, 77.7952814) - Crop: Ragi
Data fetched for point (12.7931752, 77.7952814): 15 records
Proc

Data fetched for point (12.7618545, 77.7817922): 15 records
Processing point (12.7618551, 77.7817909) - Crop: Eucalyptus
Data fetched for point (12.7618551, 77.7817909): 15 records
Processing point (12.7516752, 77.7754234) - Crop: Avare
Data fetched for point (12.7516752, 77.7754234): 15 records
Processing point (12.7516769, 77.7754255) - Crop: Avare
Data fetched for point (12.7516769, 77.7754255): 15 records
Processing point (12.7618515, 77.7817888) - Crop: Eucalyptus
Data fetched for point (12.7618515, 77.7817888): 15 records
Processing point (12.7618492, 77.7817843) - Crop: Eucalyptus
Data fetched for point (12.7618492, 77.7817843): 15 records
Processing point (12.7604392, 77.7824222) - Crop: Rose
Data fetched for point (12.7604392, 77.7824222): 15 records
Processing point (12.7232033, 77.76543) - Crop: Ragi
Data fetched for point (12.7232033, 77.76543): 15 records
Processing point (12.752315, 77.7753464) - Crop: Eucalyptus
Data fetched for point (12.752315, 77.7753464): 15 records


Data fetched for point (12.7252554, 77.7663838): 15 records
Processing point (12.7515993, 77.785304) - Crop: Ragi
Data fetched for point (12.7515993, 77.785304): 15 records
Processing point (12.753598, 77.7857172) - Crop: Cabbage
Data fetched for point (12.753598, 77.7857172): 15 records
Processing point (12.7535961, 77.7857201) - Crop: Cabbage
Data fetched for point (12.7535961, 77.7857201): 15 records
Processing point (12.7229433, 77.7731283) - Crop: Ragi
Data fetched for point (12.7229433, 77.7731283): 15 records
Processing point (12.7619222, 77.7830778) - Crop: Carrot
Data fetched for point (12.7619222, 77.7830778): 15 records
Processing point (12.7619223, 77.783078) - Crop: Carrot
Data fetched for point (12.7619223, 77.783078): 15 records
Processing point (12.7232968, 77.768214) - Crop: Avare
Data fetched for point (12.7232968, 77.768214): 15 records
Processing point (12.7509579, 77.7841027) - Crop: Cabbage
Data fetched for point (12.7509579, 77.7841027): 15 records
Processing poi

...

Data fetched for point (12.83777167, 77.79012): 15 records
Processing point (12.8366477, 77.79044) - Crop: Ragi
Data fetched for point (12.8366477, 77.79044): 15 records
Processing point (12.8366462, 77.7904448) - Crop: Ragi
Data fetched for point (12.8366462, 77.7904448): 15 records
Processing point (12.8256584, 77.7879034) - Crop: Ragi
Data fetched for point (12.8256584, 77.7879034): 15 records
Processing point (12.8329131, 77.7883362) - Crop: Ragi
Data fetched for point (12.8329131, 77.7883362): 15 records
Processing point (12.8265462, 77.7873556) - Crop: Ridgegourd
Data fetched for point (12.8265462, 77.7873556): 15 records
Processing point (12.8303033, 77.7866108) - Crop: Ragi
Data fetched for point (12.8303033, 77.7866108): 15 records
Processing point (12.8377935, 77.7914335) - Crop: Gladiolus
Data fetched for point (12.8377935, 77.7914335): 15 records
Processing point (12.837765, 77.7913717) - Crop: Gladiolus
Data fetched for point (12.837765, 77.7913717): 15 records
Processing 

Data fetched for point (12.8314292, 77.7970856): 15 records
Processing point (12.8302321, 77.8034576) - Crop: Eucalyptus
Data fetched for point (12.8302321, 77.8034576): 15 records
Processing point (12.8302262, 77.8034537) - Crop: Eucalyptus
Data fetched for point (12.8302262, 77.8034537): 15 records
Processing point (12.8335183, 77.7947233) - Crop: Rose
Data fetched for point (12.8335183, 77.7947233): 15 records
Processing point (12.8335118, 77.7947299) - Crop: Rose
Data fetched for point (12.8335118, 77.7947299): 15 records
Processing point (12.8134896, 77.8009552) - Crop: Avare
Data fetched for point (12.8134896, 77.8009552): 15 records
Processing point (12.8263508, 77.7879296) - Crop: Ragi
Data fetched for point (12.8263508, 77.7879296): 15 records
Processing point (12.8334684, 77.7947769) - Crop: Ragi
Data fetched for point (12.8334684, 77.7947769): 15 records
Processing point (12.83346833, 77.79477667) - Crop: Ragi
Data fetched for point (12.83346833, 77.79477667): 15 records
Pro

Data fetched for point (12.82842494, 77.79433471): 15 records
Processing point (12.8322445, 77.7860205) - Crop: Ragi
Data fetched for point (12.8322445, 77.7860205): 15 records
Processing point (12.8282246, 77.7868449) - Crop: Ragi
Data fetched for point (12.8282246, 77.7868449): 15 records
Processing point (12.8282331, 77.7868467) - Crop: Ragi
Data fetched for point (12.8282331, 77.7868467): 15 records
Processing point (12.8377833, 77.790105) - Crop: Ragi
Data fetched for point (12.8377833, 77.790105): 15 records
Processing point (12.8377826, 77.790104) - Crop: Ragi
Data fetched for point (12.8377826, 77.790104): 15 records
Processing point (12.8367017, 77.7904532) - Crop: Ragi
Data fetched for point (12.8367017, 77.7904532): 15 records
Processing point (12.8366983, 77.7904533) - Crop: Ragi
Data fetched for point (12.8366983, 77.7904533): 15 records
Processing point (12.8277566, 77.7986669) - Crop: Tomato
Data fetched for point (12.8277566, 77.7986669): 15 records
Processing point (12

Data fetched for point (12.8332833, 77.794875): 15 records
Processing point (12.8335283, 77.794722) - Crop: Rose
Data fetched for point (12.8335283, 77.794722): 15 records
Processing point (12.8335299, 77.7947234) - Crop: Rose
Data fetched for point (12.8335299, 77.7947234): 15 records
Processing point (12.8289048, 77.787142) - Crop: Ragi
Data fetched for point (12.8289048, 77.787142): 15 records
Processing point (12.8251024, 77.7897444) - Crop: Ragi
Data fetched for point (12.8251024, 77.7897444): 15 records
Processing point (12.8254417, 77.7867798) - Crop: Ivygourd
Data fetched for point (12.8254417, 77.7867798): 15 records
Processing point (12.825469, 77.786781) - Crop: Ivygourd
Data fetched for point (12.825469, 77.786781): 15 records
Processing point (12.8254995, 77.7868116) - Crop: Ivygourd
Data fetched for point (12.8254995, 77.7868116): 15 records
Processing point (12.8262321, 77.7880775) - Crop: Ragi
Data fetched for point (12.8262321, 77.7880775): 15 records
Processing point 

Data fetched for point (12.8262401, 77.800533): 15 records
Processing point (12.8262403, 77.800533) - Crop: Trees
Data fetched for point (12.8262403, 77.800533): 15 records
Processing point (12.8263706, 77.8073836) - Crop: Rose
Data fetched for point (12.8263706, 77.8073836): 15 records
Processing point (12.8263631, 77.8073949) - Crop: Rose
Data fetched for point (12.8263631, 77.8073949): 15 records
Processing point (12.8256554, 77.7878824) - Crop: Ragi
Data fetched for point (12.8256554, 77.7878824): 15 records
Processing point (12.8250692, 77.7897747) - Crop: Ragi
Data fetched for point (12.8250692, 77.7897747): 15 records
Processing point (12.8251068, 77.7897697) - Crop: Ragi
Data fetched for point (12.8251068, 77.7897697): 15 records
Processing point (12.8291623, 77.7929296) - Crop: Ragi
Data fetched for point (12.8291623, 77.7929296): 15 records
Processing point (12.8396294, 77.7915909) - Crop: Rose
Data fetched for point (12.8396294, 77.7915909): 15 records
Processing point (12.8

In [5]:
def fill_empty_bands(image):
  band_names = image.bandNames()
  zero_img = image.select(0).multiply(0).rename('constant').toDouble()
  zero_img_masked = zero_img.updateMask(zero_img)
  image = ee.Algorithms.If(ee.List(band_names).contains(ee.String('VV')),image, ee.Image(image).addBands(zero_img_masked.select('constant').rename('VV')))
  image = ee.Algorithms.If(ee.List(band_names).contains(ee.String('VH')),image, ee.Image(image).addBands(zero_img_masked.select('constant').rename('VH')))
  return image


def Get_S1_ImageCollections(inputStartDate, inputEndDate, roi_boundary):
  S1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
         .filter(ee.Filter.eq('instrumentMode', 'IW')) \
         .filterDate(inputStartDate, inputEndDate) \
         .filterBounds(roi_boundary)

  S1_processed = S1.map(fill_empty_bands)
  return S1_processed


def GetVV_VH_image_datewise(S1_ic):
  def get_VV_VH_datewise(date):
    zero_img = S1_ic.first().select('VV','VH').multiply(0)
    zero_img_masked = zero_img.updateMask(zero_img)

    subset_ic = S1_ic.select(['VV','VH']).filterDate(ee.Date(date), ee.Date(date).advance(6, 'day'))
    image = ee.Algorithms.If( ee.Number(subset_ic.size()).gt(0), subset_ic.mean().set('system:time_start',ee.Date(date).millis()), zero_img.set('system:time_start',ee.Date(date).millis()))

    return image
  return get_VV_VH_datewise


def Get_S1_16Day_VV_VH_TimeSeries(inputStartDate, inputEndDate, S1_ic):
  startDate = datetime.strptime(inputStartDate,"%Y-%m-%d")
  endDate = datetime.strptime(inputEndDate,"%Y-%m-%d")

  date_list = pd.date_range(start=startDate, end=endDate, freq='6D').tolist()
  date_list = ee.List( [datetime.strftime(curr_date,"%Y-%m-%d") for curr_date in date_list] )

  S1_TS =  ee.ImageCollection.fromImages(date_list.map(GetVV_VH_image_datewise(S1_ic)))
  return S1_TS


def add_sarImg_timestamp(image):
  timeImage = image.metadata('system:time_start').rename('timestamp')
  timeImageMasked = timeImage.updateMask(image.mask().select(0))
  return image.addBands(timeImageMasked)


def performInterpolation_sarTS(image):
  image = ee.Image(image)
  beforeImages = ee.List(image.get('before'))
  beforeMosaic = ee.ImageCollection.fromImages(beforeImages).mosaic()
  afterImages = ee.List(image.get('after'))
  afterMosaic = ee.ImageCollection.fromImages(afterImages).mosaic()

  # Interpolation formula
  # y = y1 + (y2-y1)*((t – t1) / (t2 – t1))
  # y = interpolated image
  # y1 = before image
  # y2 = after image
  # t = interpolation timestamp
  # t1 = before image timestamp
  # t2 = after image timestamp

  t1 = beforeMosaic.select('timestamp').rename('t1')
  t2 = afterMosaic.select('timestamp').rename('t2')
  t = image.metadata('system:time_start').rename('t')
  timeImage = ee.Image.cat([t1, t2, t])
  timeRatio = timeImage.expression('(t - t1) / (t2 - t1)', {
                  't': timeImage.select('t'),
                  't1': timeImage.select('t1'),
                  't2': timeImage.select('t2'),
              })

  interpolated = beforeMosaic.add((afterMosaic.subtract(beforeMosaic).multiply(timeRatio)))
  result = image.unmask(interpolated)

  #Saketh
  #For data points on either end of time-series
  #Before or After mosaics may still have gaps (owing to few/no images in the window)
  #Simply fill with after mosaic (for first few data points) and before mosaic (for last few datapoints)
  fill_value = ee.ImageCollection([beforeMosaic, afterMosaic]).mosaic()
  result = result.unmask(fill_value)

  return result.copyProperties(image, ['system:time_start'])


def interpolate_sar_timeseries(S1_TS):
  filtered = S1_TS.map(add_sarImg_timestamp)

  # Time window in which we are willing to look forward and backward for unmasked pixel in time series
  timeWindow = 120

  # Define a maxDifference filter to find all images within the specified days. Convert days to milliseconds.
  millis = ee.Number(timeWindow).multiply(1000*60*60*24)
  # Filter says that pick only those timestamps which lie between the 2 timestamps not more than millis difference apart
  maxDiffFilter = ee.Filter.maxDifference(
                              difference = millis,
                              leftField = 'system:time_start',
                              rightField = 'system:time_start',
                            )

  # Filter to find all images after a given image. Compare the image's timstamp against other images.
  # Images ahead of target image should have higher timestamp.
  lessEqFilter = ee.Filter.lessThanOrEquals(
                            leftField = 'system:time_start',
                            rightField = 'system:time_start'
                          )

  # Similarly define this filter to find all images before a given image
  greaterEqFilter = ee.Filter.greaterThanOrEquals(
                            leftField = 'system:time_start',
                            rightField = 'system:time_start'
                          )

  # Apply first join to find all images that are after the target image but within the timeWindow
  filter1 = ee.Filter.And( maxDiffFilter, lessEqFilter )
  join1 = ee.Join.saveAll(
                  matchesKey = 'after',
                  ordering = 'system:time_start',
                  ascending = False
          )
  join1Result = join1.apply(
                  primary = filtered,
                  secondary = filtered,
                  condition = filter1
                )

  # Apply first join to find all images that are after the target image but within the timeWindow
  filter2 = ee.Filter.And( maxDiffFilter, greaterEqFilter )
  join2 = ee.Join.saveAll(
                  matchesKey = 'before',
                  ordering = 'system:time_start',
                  ascending = True
          )
  join2Result = join2.apply(
                  primary = join1Result,
                  secondary = join1Result,
                  condition = filter2
                )

  interpolated_S1_TS = ee.ImageCollection(join2Result.map(performInterpolation_sarTS))

  return interpolated_S1_TS


In [3]:
import ee
import pandas as pd
from datetime import datetime

ee.Initialize()

# Function to read CSV and extract lat-long points
def get_points_from_csv(csv_path):
    df = pd.read_csv(csv_path)
    points = [(row['latitude'], row['longitude']) for _, row in df.iterrows()]
    return points, df

# Function to extract SAR for a single point
def extract_sar_for_point(point, start_date, end_date):
    ee_point = ee.Geometry.Point([point[1], point[0]])
    S1_ic = Get_S1_ImageCollections(start_date, end_date, ee_point)
    S1_TS = Get_S1_16Day_VV_VH_TimeSeries(start_date, end_date, S1_ic)
    interpolated_S1_TS = interpolate_sar_timeseries(S1_TS)

    # Extract SAR values as feature collection
    sar_fc = interpolated_S1_TS.map(lambda img: img.sampleRegions(
        collection=ee.FeatureCollection([ee.Feature(ee_point)]),
        scale=10
    )).flatten()

    return sar_fc

# Function to convert SAR feature collection to DataFrame
def feature_collection_to_dataframe(fc):
    # Get the feature collection as a list
    features = fc.getInfo()['features']
    data = []

    for feature in features:
        props = feature['properties']
        data.append(props)

    return pd.DataFrame(data)

# Main function to process all points and save CSV
def extract_sar_to_csv(csv_path, start_date, end_date, output_csv):
    points, df = get_points_from_csv(csv_path)
    all_data = []

    for i, point in enumerate(points):
        print(f"Processing point {i+1}/{len(points)} at {point}")
        sar_fc = extract_sar_for_point(point, start_date, end_date)
        sar_df = feature_collection_to_dataframe(sar_fc)

        # Add lat-long to SAR data
        sar_df['latitude'] = point[0]
        sar_df['longitude'] = point[1]

        all_data.append(sar_df)

    # Combine all points into a single CSV
    final_df = pd.concat(all_data, ignore_index=True)
    final_df.to_csv(output_csv, index=False)
    print(f"SAR data saved to {output_csv}")

# Example usage
csv_path = "Karnataka_Datasets/Model_Data/Karnataka_train.csv"
output_csv = "Karnataka_Datasets/Model_Data/Karnataka_SAR_Indiasat.csv"
start_date = "2021-04-01"
end_date = "2021-09-30"

extract_sar_to_csv(csv_path, start_date, end_date, output_csv)


Processing point 1/5553 at (13.0165502, 77.43563)
Processing point 2/5553 at (12.9388149, 77.4411565)
Processing point 3/5553 at (13.0170351, 77.4459952)
Processing point 4/5553 at (12.9871739, 77.3912411)
Processing point 5/5553 at (12.9944027, 77.4203926)
Processing point 6/5553 at (13.0779433, 77.5129053)
Processing point 7/5553 at (13.1369032, 77.4839342)
Processing point 8/5553 at (13.0617371, 77.4155887)
Processing point 9/5553 at (13.0198391, 77.4634807)
Processing point 10/5553 at (13.1431017, 77.438665)
Processing point 11/5553 at (13.0783995, 77.4750892)
Processing point 12/5553 at (13.1448373, 77.4321239)
Processing point 13/5553 at (12.9542286, 77.4133582)
Processing point 14/5553 at (13.1610449, 77.4552706)
Processing point 15/5553 at (13.2049643, 77.5142864)
Processing point 16/5553 at (13.22076, 77.5101033)
Processing point 17/5553 at (12.9730023, 77.3939156)
Processing point 18/5553 at (13.0784007, 77.4751346)
Processing point 19/5553 at (12.8036079, 77.5158477)
Process

Processing point 156/5553 at (13.1341539, 77.4830817)
Processing point 157/5553 at (12.9544927, 77.4529655)
Processing point 158/5553 at (13.1363428, 77.485688)
Processing point 159/5553 at (13.0701509, 77.4047684)
Processing point 160/5553 at (13.1281849, 77.4715978)
Processing point 161/5553 at (13.0518563, 77.448071)
Processing point 162/5553 at (13.0622397, 77.4054474)
Processing point 163/5553 at (13.1388022, 77.4670596)
Processing point 164/5553 at (13.2036733, 77.513985)
Processing point 165/5553 at (13.1071911, 77.4632087)
Processing point 166/5553 at (12.7826725, 77.5372769)
Processing point 167/5553 at (13.1404979, 77.4808618)
Processing point 168/5553 at (12.9533823, 77.4179073)
Processing point 169/5553 at (12.9806766, 77.3617801)
Processing point 170/5553 at (13.0142389, 77.458351)
Processing point 171/5553 at (12.9916557, 77.414559)
Processing point 172/5553 at (13.0396967, 77.448151)
Processing point 173/5553 at (13.0787895, 77.467469)
Processing point 174/5553 at (13.11

Processing point 309/5553 at (13.0709371, 77.3984094)
Processing point 310/5553 at (12.9451073, 77.3675315)
Processing point 311/5553 at (13.0617376, 77.4801433)
Processing point 312/5553 at (13.211916, 77.5556359)
Processing point 313/5553 at (13.20491, 77.5142583)
Processing point 314/5553 at (13.1370131, 77.4833957)
Processing point 315/5553 at (13.1268813, 77.4855358)
Processing point 316/5553 at (13.1271939, 77.4792514)
Processing point 317/5553 at (12.8911742, 77.4184263)
Processing point 318/5553 at (13.1011068, 77.4730091)
Processing point 319/5553 at (12.9318302, 77.4171878)
Processing point 320/5553 at (13.0419774, 77.4470319)
Processing point 321/5553 at (13.141919, 77.4858873)
Processing point 322/5553 at (13.1252693, 77.4276937)
Processing point 323/5553 at (13.1369982, 77.4813326)
Processing point 324/5553 at (13.1346231, 77.4833746)
Processing point 325/5553 at (12.9825854, 77.4005676)
Processing point 326/5553 at (13.1355463, 77.4850323)
Processing point 327/5553 at (13

Processing point 462/5553 at (13.1628717, 77.4478233)
Processing point 463/5553 at (12.8198611, 77.5234383)
Processing point 464/5553 at (13.0162517, 77.45542)
Processing point 465/5553 at (13.1387894, 77.4670528)
Processing point 466/5553 at (13.2119152, 77.5556334)
Processing point 467/5553 at (12.93574, 77.4703828)
Processing point 468/5553 at (13.0558373, 77.4047899)
Processing point 469/5553 at (13.1356746, 77.4838522)
Processing point 470/5553 at (13.0113292, 77.3828428)
Processing point 471/5553 at (12.8198501, 77.5234274)
Processing point 472/5553 at (13.1346235, 77.4833593)
Processing point 473/5553 at (12.977396, 77.4305478)
Processing point 474/5553 at (13.0639678, 77.4012553)
Processing point 475/5553 at (13.1388136, 77.4670484)
Processing point 476/5553 at (13.1344653, 77.4855041)
Processing point 477/5553 at (12.9542085, 77.4166989)
Processing point 478/5553 at (13.0165782, 77.4353274)
Processing point 479/5553 at (13.0532297, 77.4126053)
Processing point 480/5553 at (13.

Processing point 615/5553 at (13.2187256, 77.5254718)
Processing point 616/5553 at (12.8285359, 77.4609868)
Processing point 617/5553 at (12.7246133, 77.6781933)
Processing point 618/5553 at (12.9133256, 77.7925836)
Processing point 619/5553 at (12.8081756, 77.6060751)
Processing point 620/5553 at (13.049535, 77.4510567)
Processing point 621/5553 at (12.8063503, 77.7342066)
Processing point 622/5553 at (13.1130118, 77.4794841)
Processing point 623/5553 at (12.7917938, 77.4763649)
Processing point 624/5553 at (13.0506395, 77.4213392)
Processing point 625/5553 at (12.9791766, 77.3719263)
Processing point 626/5553 at (12.8924833, 77.3568038)
Processing point 627/5553 at (13.0019199, 77.4268953)
Processing point 628/5553 at (13.1371318, 77.4875626)
Processing point 629/5553 at (12.8966753, 77.7709668)
Processing point 630/5553 at (12.7431815, 77.7208617)
Processing point 631/5553 at (12.9557319, 77.4118108)
Processing point 632/5553 at (13.0673271, 77.4428605)
Processing point 633/5553 at 

...

Processing point 4827/5553 at (13.2256549, 77.4993432)
Processing point 4828/5553 at (12.7699384, 77.53968)
Processing point 4829/5553 at (13.1821406, 77.709604)
Processing point 4830/5553 at (13.15479, 77.4560183)
Processing point 4831/5553 at (12.7553502, 77.7917622)
Processing point 4832/5553 at (13.1573185, 77.604565)
Processing point 4833/5553 at (12.7168653, 77.7267902)
Processing point 4834/5553 at (12.66846, 77.7079983)
Processing point 4835/5553 at (13.0937073, 77.7559682)
Processing point 4836/5553 at (12.7600669, 77.6446899)
Processing point 4837/5553 at (12.7245929, 77.765582)
Processing point 4838/5553 at (13.1786595, 77.6931023)
Processing point 4839/5553 at (13.1809801, 77.5910038)
Processing point 4840/5553 at (13.1095427, 77.7536188)
Processing point 4841/5553 at (12.7191144, 77.7529343)
Processing point 4842/5553 at (13.0876301, 77.7122877)
Processing point 4843/5553 at (12.9491699, 77.7649578)
Processing point 4844/5553 at (13.2133083, 77.5495067)
Processing point 48

Processing point 4977/5553 at (12.927485, 77.3767291)
Processing point 4978/5553 at (12.9132303, 77.7870205)
Processing point 4979/5553 at (13.1701354, 77.6901962)
Processing point 4980/5553 at (12.8228441, 77.7940509)
Processing point 4981/5553 at (12.8945046, 77.3647483)
Processing point 4982/5553 at (12.8456947, 77.703094)
Processing point 4983/5553 at (12.70872, 77.7540817)
Processing point 4984/5553 at (13.0860498, 77.6872563)
Processing point 4985/5553 at (12.7880136, 77.728265)
Processing point 4986/5553 at (12.8100519, 77.4627667)
Processing point 4987/5553 at (13.0434472, 77.7704461)
Processing point 4988/5553 at (12.8219948, 77.4478031)
Processing point 4989/5553 at (12.8481827, 77.4476992)
Processing point 4990/5553 at (12.8919476, 77.360277)
Processing point 4991/5553 at (13.0982317, 77.4737738)
Processing point 4992/5553 at (12.9753058, 77.4187946)
Processing point 4993/5553 at (13.2215166, 77.499809)
Processing point 4994/5553 at (12.8324444, 77.7686383)
Processing point 

Processing point 5128/5553 at (12.9131023, 77.4240725)
Processing point 5129/5553 at (12.7989269, 77.8041828)
Processing point 5130/5553 at (12.8919467, 77.3602933)
Processing point 5131/5553 at (13.187343, 77.5131039)
Processing point 5132/5553 at (12.9395042, 77.3740027)
Processing point 5133/5553 at (12.9812744, 77.4199573)
Processing point 5134/5553 at (13.1092892, 77.4313868)
Processing point 5135/5553 at (12.7575569, 77.7562339)
Processing point 5136/5553 at (12.8962087, 77.4147045)
Processing point 5137/5553 at (13.039102, 77.7725095)
Processing point 5138/5553 at (12.8323033, 77.423485)
Processing point 5139/5553 at (12.9401201, 77.3701917)
Processing point 5140/5553 at (12.8306102, 77.750673)
Processing point 5141/5553 at (12.971805, 77.4488917)
Processing point 5142/5553 at (12.7551689, 77.646994)
Processing point 5143/5553 at (13.1340103, 77.7193265)
Processing point 5144/5553 at (12.7280693, 77.6352031)
Processing point 5145/5553 at (13.0966614, 77.4918804)
Processing point

Processing point 5278/5553 at (12.9387631, 77.3815996)
Processing point 5279/5553 at (12.9174152, 77.418999)
Processing point 5280/5553 at (13.0694663, 77.7566952)
Processing point 5281/5553 at (12.8919567, 77.360315)
Processing point 5282/5553 at (12.9829465, 77.4242556)
Processing point 5283/5553 at (12.8161433, 77.71046)
Processing point 5284/5553 at (12.8100337, 77.4627559)
Processing point 5285/5553 at (12.8192681, 77.5915529)
Processing point 5286/5553 at (12.803272, 77.7945962)
Processing point 5287/5553 at (12.69457, 77.6334435)
Processing point 5288/5553 at (12.9887809, 77.3826864)
Processing point 5289/5553 at (13.1643326, 77.5040049)
Processing point 5290/5553 at (12.8245809, 77.4525434)
Processing point 5291/5553 at (12.833225, 77.4264101)
Processing point 5292/5553 at (12.9861419, 77.3728769)
Processing point 5293/5553 at (12.8658266, 77.4630059)
Processing point 5294/5553 at (13.184912, 77.6764116)
Processing point 5295/5553 at (12.8092078, 77.7734125)
Processing point 52

Processing point 5428/5553 at (12.7708283, 77.6577914)
Processing point 5429/5553 at (13.2170046, 77.5083881)
Processing point 5430/5553 at (12.7193633, 77.7629)
Processing point 5431/5553 at (13.1249398, 77.4279531)
Processing point 5432/5553 at (12.9780375, 77.4241293)
Processing point 5433/5553 at (13.1471679, 77.7219455)
Processing point 5434/5553 at (13.038048, 77.4100864)
Processing point 5435/5553 at (12.8608672, 77.8142152)
Processing point 5436/5553 at (12.7500658, 77.5053335)
Processing point 5437/5553 at (12.9282426, 77.3831845)
Processing point 5438/5553 at (12.748725, 77.6652133)
Processing point 5439/5553 at (12.7835139, 77.6209026)
Processing point 5440/5553 at (12.7162121, 77.6784979)
Processing point 5441/5553 at (12.8332283, 77.4264117)
Processing point 5442/5553 at (12.832799, 77.7490917)
Processing point 5443/5553 at (12.8242658, 77.78727)
Processing point 5444/5553 at (12.7981909, 77.79289)
Processing point 5445/5553 at (12.8323078, 77.4235083)
Processing point 544

In [3]:
import pandas as pd

# Load the CSV file
file_path = "Karnataka_Datasets/Model_Data/final_train_height.csv"  # Replace with your actual file path
df = pd.read_csv(file_path)

# Rename columns
df.columns = [f"{col.split('_')[1]}_{col.split('_')[0]}" if "_" in col else col for col in df.columns]

# Save back to the same CSV file
df.to_csv(file_path, index=False)

print("Column names updated and file saved successfully!")


Column names updated and file saved successfully!


In [5]:
import pandas as pd

# Load the CSV file
file_path = "Karnataka_Datasets/Model_Data/final_train_height.csv"  # Replace with your actual file path
df = pd.read_csv(file_path)

# Define mapping for height categories
height_mapping = {"low": 0, "medium": 1, "high": 2}

# Create a new column 'height_numeric'
df["height_numeric"] = df["height"].map(height_mapping)

# Save back to the CSV file
df.to_csv(file_path, index=False)

print("New column 'height_numeric' added successfully!")


New column 'height_numeric' added successfully!
